## Necessary imports

In [81]:
#import necessary stuff
import pandas as pd
import requests
import os
import json

## Use the JSON file containing Google Trends locations

Retrieved from this link: https://trends.google.com/trends/api/explore/pickers/geo?hl=en-US

In [89]:
google_trends_locations_file_reader = open('resources/google_trends-locations.json', 'r')
google_trends_locations = json.load(google_trends_locations_file_reader)

## Focus only on the locations in the US

In [90]:
def get_subtree_with_name(json_obj, name):
    if json_obj['name'] == name:
        return json_obj
    elif 'children' in json_obj:
        for child in json_obj['children']:
            subtree = get_subtree_with_name(child, name)
            if subtree is not None:
                return subtree

In [91]:
only_US = get_subtree_with_name(google_trends_locations, 'United States')
# save it
with open('resources/google_trends-locations-US.json', 'w') as outfile:
    json.dump(only_US, outfile)

## Focus only on the Metro Areas

In [92]:
class MetroArea:
    def __init__(self, name, state_id, self_id):
        self.name = name
        self.state_id = state_id
        self.id = self_id

    def __eq__(self, other):
        return self.id == other.id

    def __hash__(self):
        return self.id

    def __str__(self):
        return f'(name: {self.name}, id: {self.id})'

    def __repr__(self):
        return self.__str__()

In [107]:
# These are critical locations that only has one metro area, but the metro area name is not the same as the state name
google_trends_critical_locations = {
    'Utah': ('Salt Lake City UT', 'WY', 770),
    'District of Columbia': ('Washington DC (Hagerstown MD)', 'WV', 511),
    'Rhode Island': ('Providence RI-New Bedford MA', 'MA', 521),
}

In [108]:
def get_leaves_as_tuples(json_obj, state_id):
    if 'children' not in json_obj:
        if state_id != 'US':
            return [MetroArea(json_obj['name'], state_id, int(json_obj['id']))]
        return [MetroArea(*google_trends_critical_locations[json_obj['name']])]
    else:
        leaves = []
        for child in json_obj['children']:
            leaves += get_leaves_as_tuples(child, json_obj['id'])
        return leaves

In [109]:
leaves = get_leaves_as_tuples(only_US, None)
leaves = list(set(leaves))
metro_areas_table = pd.DataFrame([leave.__dict__ for leave in leaves])
metro_areas_table

,name,state_id,id
0,Baltimore MD,MD,512
1,Flint-Saginaw-Bay City MI,MI,513
2,Buffalo NY,NY,514
3,Cincinnati OH,IN,515
4,Erie PA,PA,516
...,...,...,...
205,Savannah GA,GA,507
206,Pittsburgh PA,PA,508
207,Ft. Wayne IN,IN,509
208,Cleveland-Akron (Canton) OH,OH,510


## Match the counties with metro areas and find median household income per metro area

### Load the county correspondence table
The correspondence table is retrieved from this link: https://sites.google.com/view/jacob-schneider/resources

In [110]:
# read the csv
correspondence_table = pd.read_csv('resources/trends_metro_counties_crosswalk.csv')
correspondence_table

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,county_name,trends_geocode,trends_geoname,split_county,trends_geocode2,trends_geoname2
0,1,1,161526,0500000US01001,1001,Autauga,698,Montgomery (Selma) AL,0,,
1,1,3,161527,0500000US01003,1003,Baldwin,686,Mobile AL-Pensacola (Ft. Walton Beach) FL,0,,
2,1,5,161528,0500000US01005,1005,Barbour,522,Columbus GA,0,,
3,1,7,161529,0500000US01007,1007,Bibb,630,Birmingham AL,0,,
4,1,9,161530,0500000US01009,1009,Blount,630,Birmingham AL,0,,
...,...,...,...,...,...,...,...,...,...,...,...
3137,56,37,1609192,0500000US56037,56037,Sweetwater,770,Salt Lake City UT,0,,
3138,56,39,1605083,0500000US56039,56039,Teton,758,Idaho Falls-Pocatello ID,0,,
3139,56,41,1605084,0500000US56041,56041,Uinta,770,Salt Lake City UT,0,,
3140,56,43,1605085,0500000US56043,56043,Washakie,767,Casper-Riverton WY,0,,


### Load the census data tables for different years

In [112]:
one_year_tables = {}
one_year_supp_tables = {}
five_year_tables = {}

for year in range(2011, 2022):
    # try loading one year table
    try:
        one_year_tables[year] = pd.read_csv(f'../census-data/data/one-year-estimates/{year}.csv')
    except FileNotFoundError:
        print(f'No one year table for {year}')
    
    # try loading one year supplemental table
    try:
        one_year_supp_tables[year] = pd.read_csv(f'../census-data/data/one-year-supplement-estimates/{year}.csv')
    except FileNotFoundError:
        print(f'No one year supplemental table for {year}')
    
    # try loading five year table
    try:
        five_year_tables[year] = pd.read_csv(f'../census-data/data/five-year-estimates/{year}.csv')
    except FileNotFoundError:
        print(f'No five year table for {year}')

No one year supplemental table for 2011
No one year supplemental table for 2012
No one year supplemental table for 2013
No one year table for 2020
No one year supplemental table for 2020
No five year table for 2021


### Find the median household income per metro area

In [132]:
# we are assuming uniform distribution of the income in a given bin/interval
# we could also apply more sophisticated methods (E.g. https://journals.sagepub.com/doi/epub/10.1177/0081175015599807)
def interpolate_median(intervals):

    # convert a description of an interval to a valid range
    def get_range(desc):
        if desc == '200000 or more':
            return (200000, 700000) # let's fix the upperbound as something resonable
        else:
            return tuple(map(int, desc.replace(' ', '').split('-')))

    # find the mapping of the candidate median inside a given range
    def locate(m, interval_range):
        return max(0, min(1, (m - interval_range[0]) / (interval_range[1] - interval_range[0])))

    def get_cnt_smaller(m, intervals_with_range):
        return sum([
            max(0, locate(m, interval_range) * pop_cnt)
            for interval_range, pop_cnt in intervals_with_range
        ])

    # convert intervals with descriptions into intervals with ranges
    intervals_with_range = [
        (get_range(desc), pop_cnt)
        for desc, pop_cnt in intervals.items()
    ]

    # all the households in the given intervals
    cnt_all = sum([pop_cnt for _, pop_cnt in intervals_with_range])

    # if the total number of households is 0, then we return -1
    if cnt_all == 0:
        return -1

    # do binary search to find the median
    l, r = 0, 700001
    for _ in range(30):
        m = (l + r) / 2
        # find the number of households with income less than or equal to m
        cnt_smaller = get_cnt_smaller(m, intervals_with_range)
        # if our candidate median should be larger
        if cnt_smaller < cnt_all / 2:
            l = m
        # or should be smaller
        else:
            r = m

    # return the approximated median
    return round((l + r) / 2, 2)

def update_all_intervals(all_intervals, county_intervals, divide):
    for interval in county_intervals:
        interval_desc, interval_pop_cnt = interval
        if not interval_desc in all_intervals:
            all_intervals[interval_desc] = 0
        # if divide is 1, then we split the population evenly
        all_intervals[interval_desc] += interval_pop_cnt / (1 + divide) 
    return all_intervals

def handle_rounding(val):
    try:
        return round(val)
    except:
        return 0

def get_intervals_for_county(table, affgeoid):
    county_row = table[table['Geo_id'] == affgeoid]
    interest = county_row.iloc[:, 2:]
    intervals = [
        (col, handle_rounding(county_row[col].values[0]))
        for col in interest.columns
    ]
    return intervals

In [157]:
def get_median_income_interpolation(metro_row, year, table_finder_func):
    
    metro_id = metro_row['id']
    counties = correspondence_table[
        (correspondence_table['trends_geocode'] == metro_id) | 
        (correspondence_table['trends_geocode2'] == f'{metro_id}')
    ]

    all_intervals = {}
    for _, county_row in counties.iterrows():
    
        affgeoid = county_row['AFFGEOID']

        table = table_finder_func(affgeoid, year)

        if table is None:
            continue

        county_intervals = get_intervals_for_county(table, affgeoid)
        # here, we set divide parameter to 1 if the county is split into two metro areas
        # since we do not know how much of the county belongs to each metro area
        # we assume that the county is split evenly
        all_intervals = update_all_intervals(all_intervals, county_intervals, county_row['split_county'] == 1)
    
    return interpolate_median(all_intervals)

### Compute different types of tables

1. One table that only uses 5 year estimates (for years 2011 - 2020). If no datas exists for that county, skips its contribution to a metro area.
2. One table that tries to use 1 year estimates for a county. If it does not exist, then checks 5 year estimates. If it does not exist too, then ignores that county (for years 2014 - 2019).
3. One table that tries to use 1 year estimates for a county. If it does not exist, then checks 5 year estimates. If it does not exist too, then retrieves the estimate from the nearest year (preferrably 1 year estimate) (for years 2014 - 2019 and 2021).

#### Table 1:

In [158]:
def find_the_table_1(affgeoid, year):
    if year in five_year_tables and affgeoid in five_year_tables[year]['Geo_id'].values:
        return five_year_tables[year]        
    return None

In [159]:
metro_areas_table_1 = metro_areas_table.copy()
for year in range(2011, 2021):
    metro_areas_table_1[f'median_income_{year}'] = [
        get_median_income_interpolation(metro_row, year, find_the_table_1)
        for _, metro_row in metro_areas_table.iterrows()
    ]
    print(f'Finished {year}')
metro_areas_table_1

Finished 2011
Finished 2012
Finished 2013
Finished 2014
Finished 2015
Finished 2016
Finished 2017
Finished 2018
Finished 2019
Finished 2020


,name,state_id,id,median_income_2011,median_income_2012,median_income_2013,median_income_2014,median_income_2015,median_income_2016,median_income_2017,median_income_2018,median_income_2019,median_income_2020
0,Baltimore MD,MD,512,68391.04,68928.28,69424.24,70117.38,70638.12,72217.83,74800.76,77993.55,81272.49,83348.86
1,Flint-Saginaw-Bay City MI,MI,513,43836.32,43635.57,43021.42,43496.56,43838.59,44553.31,46042.25,47740.96,49142.99,50455.48
2,Buffalo NY,NY,514,47179.47,48039.91,48387.03,48968.55,49233.45,50591.79,51997.30,54285.69,56389.05,57887.94
3,Cincinnati OH,IN,515,54131.45,54211.58,54087.13,54643.44,55090.73,56518.81,58975.13,61219.15,63838.19,66076.05
4,Erie PA,PA,516,43991.54,44754.73,44871.36,45493.37,45977.75,46963.67,47976.93,49433.01,51341.98,52756.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Savannah GA,GA,507,45743.44,45795.87,46228.28,46854.60,46621.46,48002.91,49411.38,51703.77,54805.73,57239.44
206,Pittsburgh PA,PA,508,48194.87,49020.97,49788.92,50976.72,51807.61,53311.18,55399.18,57546.07,59780.57,61292.10
207,Ft. Wayne IN,IN,509,48360.87,48306.87,48147.01,48751.61,48914.46,49644.82,51569.39,54120.76,56076.32,57854.98
208,Cleveland-Akron (Canton) OH,OH,510,48159.08,48149.34,48255.67,48778.50,49237.08,50329.56,52178.02,54425.92,56288.29,57766.94


#### Table 2:

In [160]:
def find_the_table_2(affgeoid, year):
    if year in one_year_tables and affgeoid in one_year_tables[year]['Geo_id'].values:
        return one_year_tables[year]
    if year in one_year_supp_tables and affgeoid in one_year_supp_tables[year]['Geo_id'].values:
        return one_year_supp_tables[year]
    if year in five_year_tables and affgeoid in five_year_tables[year]['Geo_id'].values:
        return five_year_tables[year]
    return None

In [161]:
metro_areas_table_2 = metro_areas_table.copy()
for year in range(2014, 2020):
    metro_areas_table_2[f'median_income_{year}'] = [
        get_median_income_interpolation(metro_row, year, find_the_table_2)
        for _, metro_row in metro_areas_table.iterrows()
    ]
    print(f'Finished {year}')
metro_areas_table_2

Finished 2014
Finished 2015
Finished 2016
Finished 2017
Finished 2018
Finished 2019


,name,state_id,id,median_income_2014,median_income_2015,median_income_2016,median_income_2017,median_income_2018,median_income_2019
0,Baltimore MD,MD,512,70881.88,72499.03,75956.00,77210.39,79958.06,83104.75
1,Flint-Saginaw-Bay City MI,MI,513,44103.40,45619.89,45651.65,46857.29,49075.39,50653.71
2,Buffalo NY,NY,514,48958.45,50666.41,52488.76,53664.79,55573.31,58448.36
3,Cincinnati OH,IN,515,55259.99,56563.07,59639.58,61340.97,62990.00,67120.59
4,Erie PA,PA,516,45798.81,47960.91,47834.63,50582.48,49778.06,51761.04
...,...,...,...,...,...,...,...,...,...
205,Savannah GA,GA,507,47576.84,46683.00,52067.77,50983.98,53467.19,57428.38
206,Pittsburgh PA,PA,508,51734.86,53680.40,55406.80,57318.77,58715.51,62454.03
207,Ft. Wayne IN,IN,509,49170.14,50226.31,51792.31,53472.20,56923.49,57731.06
208,Cleveland-Akron (Canton) OH,OH,510,49169.54,50959.39,52063.25,53579.07,56584.84,57389.25


#### Table 3:

In [162]:
def find_the_table_3(affgeoid, year):
    for try_year in range(year, 2013, -1):
        if try_year in one_year_tables and affgeoid in one_year_tables[try_year]['Geo_id'].values:
            return one_year_tables[try_year]
        if try_year in one_year_supp_tables and affgeoid in one_year_supp_tables[try_year]['Geo_id'].values:
            return one_year_supp_tables[try_year]
        if try_year in five_year_tables and affgeoid in five_year_tables[try_year]['Geo_id'].values:
            return five_year_tables[try_year]
    return None

In [163]:
metro_areas_table_3 = metro_areas_table.copy()
for year in list(range(2014, 2020)) + [2021]:
    metro_areas_table_3[f'median_income_{year}'] = [
        get_median_income_interpolation(metro_row, year, find_the_table_3)
        for _, metro_row in metro_areas_table.iterrows()
    ]
    print(f'Finished {year}')
metro_areas_table_3

Finished 2014
Finished 2015
Finished 2016
Finished 2017
Finished 2018
Finished 2019
Finished 2021


,name,state_id,id,median_income_2014,median_income_2015,median_income_2016,median_income_2017,median_income_2018,median_income_2019,median_income_2021
0,Baltimore MD,MD,512,70881.88,72499.03,75956.00,77210.39,79958.06,83104.75,85820.27
1,Flint-Saginaw-Bay City MI,MI,513,44103.40,45619.89,45651.65,46857.29,49075.39,50653.71,53514.59
2,Buffalo NY,NY,514,48958.45,50666.41,52488.76,53664.79,55573.31,58448.36,61238.27
3,Cincinnati OH,IN,515,55259.99,56563.07,59639.58,61340.97,62990.00,67120.59,70220.94
4,Erie PA,PA,516,45798.81,47960.91,47834.63,50582.48,49778.06,51761.04,56730.36
...,...,...,...,...,...,...,...,...,...,...
205,Savannah GA,GA,507,47576.84,46683.00,52067.77,50983.98,53467.19,57428.38,61071.21
206,Pittsburgh PA,PA,508,51734.86,53680.40,55406.80,57318.77,58715.51,62454.03,65944.17
207,Ft. Wayne IN,IN,509,49170.14,50226.31,51792.31,53472.20,56923.49,57731.06,62515.75
208,Cleveland-Akron (Canton) OH,OH,510,49169.54,50959.39,52063.25,53579.07,56584.84,57389.25,61762.68


## Save the tables

In [164]:
# save the metro areas table to the data dir
if not os.path.exists('data'):
    os.mkdir('data')

metro_areas_table_1.to_csv('data/metro_areas_table_1.csv', index=False)
metro_areas_table_2.to_csv('data/metro_areas_table_2.csv', index=False)
metro_areas_table_3.to_csv('data/metro_areas_table_3.csv', index=False)